### OBJECTIVE:
#### - Number of individuals with RI DB PRODUCTION, who have FB or MyA contract and have payment preferences: grouped by: paymentGroupCode/paymentGroupName

##### 1- imports section

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

C:\Users\t356726\AppData\Local\Temp\ipykernel_23232\2027920682.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Initialization of a Spark Session
sc = SparkSession.builder.appName("PySparkThiernoPP")\
    .config ("spark.sql.shuffle.partitions", "50") \
    .config("spark.driver.maxResultSize","10g") \
    .config ("spark.sql.execution.arrow.enabled", "true")\
    .getOrCreate()

##### 2- import CSV Input files

In [3]:
#CSV FILES
df_roleContrats = sc.read.options(header=True).csv("C:\\Users\\t356726\\Desktop\\extracts\\PRD\\extract_role_contrat_PRD2.csv")
df_repindPP = sc.read.options(header=True).csv('C:\\Users\\t356726\\Desktop\\extracts\\PRD\\export_repind_pp.csv')

In [4]:
print( "total repind_PP export : " + str(df_repindPP.count()) )
print( "total role_contracts export : " + str(df_roleContrats.count()) )

total repind_PP export : 8236357
total role_contracts export : 25338740


In [5]:
df_repindPP.show(5)
df_roleContrats.show(5)

+------------+------------+-------------+
|         GIN|PAYMENTGROUP|PAYMENTMETHOD|
+------------+------------+-------------+
|400319584754|          CC|           MC|
|400403812781|          CC|       FBAMEX|
|400288531262|          CC|         VISA|
|400502390443|          CC|         VISA|
|800015313951|          DC|        MC_DC|
+------------+------------+-------------+
only showing top 5 rows

+------------+
|        SGIN|
+------------+
|400783008016|
|400403877100|
|400591432440|
|400000217056|
|400702581796|
+------------+
only showing top 5 rows



##### 3- merge both csv files REPIND_PP && ROLE_CONTRATS

In [6]:
joined_df = df_roleContrats.join(df_repindPP, df_roleContrats.SGIN == df_repindPP.GIN, "inner")\
                            .select("GIN", "PAYMENTGROUP", "PAYMENTMETHOD")
joined_df.show(10)

+------------+------------+-------------+
|         GIN|PAYMENTGROUP|PAYMENTMETHOD|
+------------+------------+-------------+
|000000002563|          CC|         VISA|
|000000003565|          CC|         VISA|
|000000003720|          CC|         VISA|
|000000003720|          CC|         VISA|
|000000003720|          CC|         VISA|
|000000003720|          CC|         VISA|
|000000013015|          CC|           MC|
|000000013015|          CC|         VISA|
|000000017860|          CC|       FBAMEX|
|000000017963|          CC|         VISA|
+------------+------------+-------------+
only showing top 10 rows



##### 4- Group results by "PAYMENTGROUP" and "PAYMENTMETHOD"

In [10]:
sorted_grouped_df = joined_df.groupBy("PAYMENTGROUP", "PAYMENTMETHOD")\
                        .count() \
                        .sort("PAYMENTGROUP")


In [11]:
sorted_grouped_df.show(5)

+------------+-------------+-------+
|PAYMENTGROUP|PAYMENTMETHOD|  count|
+------------+-------------+-------+
|          BT|           BT|  55957|
|          CC|       CBC-MC|   2499|
|          CC|         AMEX| 894408|
|          CC|         UATP|    429|
|          CC|         VISA|2977437|
+------------+-------------+-------+
only showing top 5 rows



##### 5- Create new Table to retrieve total count by PAYMENTGROUP and PAYMENTMETHOD

In [16]:
df_repindPP_count = df_repindPP.groupBy("PAYMENTGROUP", "PAYMENTMETHOD").count()
df_repindPP_count = df_repindPP_count.withColumnRenamed("count", "TOTAL_COUNT")
df_repindPP_count.show(5)

+------------+-------------+-----------+
|PAYMENTGROUP|PAYMENTMETHOD|TOTAL_COUNT|
+------------+-------------+-----------+
|         OLB|    CELLULANT|         13|
|         OLB|        ENETS|       1006|
|     EWALLET|       ALIPAY|        611|
|          CC|         VISA|    3701073|
|          CC|   IDEAL_RABO|          3|
+------------+-------------+-----------+
only showing top 5 rows



##### 6- Reapply merge between both tables 

In [15]:
final_result = sorted_grouped_df.join(df_repindPP_count, ["PAYMENTGROUP","PAYMENTMETHOD"] , "inner")\
                            .select("PAYMENTGROUP","PAYMENTMETHOD","TOTAL_COUNT","count")
final_result.show(5)

+------------+----------------+-----------+-------+
|PAYMENTGROUP|   PAYMENTMETHOD|TOTAL_COUNT|  count|
+------------+----------------+-----------+-------+
|         OLB|       CELLULANT|         13|     13|
|         OLB|           ENETS|       1006|    560|
|     EWALLET|          ALIPAY|        611|    598|
|          LB|AW_CAB_ARUBABANK|         60|     57|
|          CC|            VISA|    3701073|2977437|
+------------+----------------+-----------+-------+
only showing top 5 rows



#### NB:
- export table to txt didn't work for me, so i just printed all lines of result table (~80 lines) and then send report by email. 